# #CUDA C Best Practices Guide -- Shared Memory Optimisations

Ensure the binary is up to date

In [4]:
! make

make: Nothing to be done for 'all'.


Run experiment and collect the tiny runtime data

In [5]:
! ./sbd ./gpu-strided-global-memory-access-unoptimised.cl tiny 0 0 runtime

***** LSB_Init >gpu_memory_access< writing to >lsb.gpu_memory_access.r0< *****
Attempting kernel: ./gpu-strided-global-memory-access-unoptimised.cl with contents:
//shared memory in matrix multiplication ported from the [cuda c best practices guide](https://docs.nvidia.com/cuda/cuda-c-best-practices-guide/index.html#shared-memory-in-matrix-multiplication-c-aa)

__kernel void simpleMultiply(__global float *a,__global float *b,__global float *c, int N)
{
    int row = get_group_id(1) * get_local_size(1) + get_local_id(1);//blockIdx.y * blockDim.y + threadIdx.y;
    int col = get_group_id(0) * get_local_size(0) + get_local_id(0);//blockIdx.x * blockDim.x + threadIdx.x;
    float sum = 0.0f;
    for (int i = 0; i < TILE_DIM; i++) {
        sum += a[row*TILE_DIM+i] * b[i*N+col];
    }
    c[row*TILE_DIM+col] = sum;
}

__kernel void coalescedMultiply(__global float* a, __global float* b, __global float* c, int N)
{
    __local float aTile[TILE_DIM][TILE_DIM];
    __local float bTile[TILE_DIM

Analyse the runtimes (to ensure the OpenCL port of the cuda codes matches those shown in their paper)

Generate a representation of the AIWC feature-space changing between kernels